In [46]:
import pandas as pd
import numpy as np
import os
import sys
from tqdm import tqdm

### Read the data

In [47]:
current_path = os.getcwd()
day = int(current_path.split('day')[1])

fn = 'day' + str(day) + '.txt'

file_content = open(fn).read().split('\n')

test_fn = 'day' + str(day) + 'test.txt'

test_file_content = open(test_fn).read().split('\n')

### Part 1

In [49]:
class WiringDiagram:
    def __init__(self, diagram_instructions):
        self.components = list()
        self.parse_diagram(diagram_instructions)
        self.connections = self.get_connections()

    def get_connections(self):
        connections = list()
        for component in self.components:
            for other_component in component.connected_components:
                if (component, other_component) not in connections and (other_component, component) not in connections:
                    connections.append((component, other_component))
        return connections

    def parse_diagram(self, diagram_instructions):
        for row in diagram_instructions:
            first_name, other_names = row.split(': ')
            other_names = other_names.split(' ')
            first_component = self.create_or_get_component(first_name)
            for other_name in other_names:
                other_component = self.create_or_get_component(other_name)
                first_component.add_connection(other_component)
                other_component.add_connection(first_component)
                self.add_component(other_component)
            self.add_component(first_component)

    def create_or_get_component(self, name):
        for component in self.components:
            if component.name == name:
                return component
        return Component(name)

    def add_component(self, component):
        if component not in self.components:
            self.components.append(component)

    def remove_three_connections_to_break_circuit(self):
        for i, connection_i in tqdm(enumerate(self.connections[:-2])):
            # only iterate after i
            for j, connection_j in enumerate(self.connections[i+1:-1]):
                # only iterate after j
                for connection_k in self.connections[j+1:]:
                    # check if circuit is broken
                    if not self.check_full_circuit(removed_connections=[connection_i, connection_j, connection_k]):
                        
                        component_i0, component_i1 = connection_i
                        size_group_0 = self.get_circuit_size(component_i0, removed_connections=[connection_i, connection_j, connection_k])
                        size_group_1 = self.get_circuit_size(component_i1, removed_connections=[connection_i, connection_j, connection_k])
                        print('Circuit is broken by removing', connection_i, connection_j, connection_k)
                        print('Size of circuit 0:', size_group_0)
                        print('Size of circuit 1:', size_group_1)
                        print('Product:', size_group_0 * size_group_1)
                        return connection_i, connection_j, connection_k
                    
    def get_circuit_size(self, component, removed_connections=None):
        size = 1
        unvisited_components = self.components.copy()
        unvisited_components.remove(component)
        to_visit = self.get_connected_components(component, removed_connections=removed_connections)

        while len(unvisited_components) > 0 and len(to_visit) > 0:
            current_component = to_visit.pop(0)
            if current_component in unvisited_components:
                unvisited_components.remove(current_component)
                size += 1
                for component in self.get_connected_components(current_component, removed_connections=removed_connections):
                    if component not in to_visit and component in unvisited_components:
                        to_visit.append(component)
        return size

    def get_connected_components(self, component, removed_connections=None):
        connected_components = list()
        for other_component in component.connected_components:
            if removed_connections is None or (component, other_component) not in removed_connections and (other_component, component) not in removed_connections:
                connected_components.append(other_component)
        return connected_components

    def check_full_circuit(self, removed_connections=None):
        unvisited_components = self.components.copy()
        # return whether every component is connected to every other component (directly or indirectly)
        # start with first component
        first_component = self.components[0]
        # check for every other component
        unvisited_components.remove(first_component)
        to_visit = self.get_connected_components(first_component, removed_connections=removed_connections)

        while len(unvisited_components) > 0 and len(to_visit) > 0:
            current_component = to_visit.pop(0)
            if current_component in unvisited_components:
                unvisited_components.remove(current_component)
                for component in self.get_connected_components(current_component, removed_connections=removed_connections):
                    if component not in to_visit and component in unvisited_components:
                        to_visit.append(component)
        
        if len(unvisited_components) == 0:
            return True
        if len(to_visit) == 0:
            print(unvisited_components)
            return False


    def __repr__(self):
        return_str = ''
        for component in self.components:
            return_str += str(component) + '\n'
        return return_str


class Component:
    def __init__(self, name):
        self.name = name
        self.connected_components = list()
    
    def add_connection(self, component):
        # check if component is already in connected_components
        if component not in self.connected_components:
            self.connected_components.append(component)
    
    def __eq__(self, other):
        return self.name == other.name

    def __repr__(self):
        return_str = self.name# + ': '
        # for component in self.connected_components:
        #     return_str += component.name + ' '
        return return_str

In [51]:
working_file = file_content

wiring_diagram = WiringDiagram(working_file)

wiring_diagram.remove_three_connections_to_break_circuit()

0it [00:00, ?it/s]

In [ ]:
import collections as C


G = C.defaultdict(set)

for line in open('day25.txt'):
    u, *vs = line.replace(':','').split()
    for v in vs: G[u].add(v); G[v].add(u)


S = set(G)

count = lambda v: len(G[v]-S)

while sum(map(count, S)) != 3:
    S.remove(max(S, key=count))

print(len(S) * len(set(G)-S))
